

これは[『Haskellによる並列・並行プログラミング』リモート輪講 #1](https://eigenspace.connpass.com/event/172164/)で学んだことのメモです。

[`:sprint`](https://downloads.haskell.org/~ghc/latest/docs/html/users_guide/ghci.html#ghci-cmd-:sprint)は変数の評価を"行わずに"その内容を表示する機能。これを使えばサンクのまま詰まれていてまだ評価されていない部分も確認することができる

In [1]:
let x = 1 + 2 :: Int

:sprint x

x = _

↑アンダースコアはサンクを表しておりxがまだ評価されていないことが分かる

In [2]:
x

:sprint x

3

x = 3

↑`x`が評価されたので中身も見えるようになった

次は変数が変数を参照している例

In [3]:
x = 1 + 2 :: Int
y = x + 1

:sprint x
:sprint y

x = _

y = _

In [4]:
seq y ()

:sprint x
:sprint y

()

x = 3

y = 4

↑`seq`を使うと第一引数が弱頭部正規形（最初の構成子が見えるところ）まで強制的に評価される  

In [5]:
x = 1 + 2 :: Int
z = (x, x)

:sprint z

z = _

あれ？これはへいへいHaskellでは `z = (_, _)` と見えるはずなんだけど違う結果になってしまった。

In [6]:
import Data.Tuple

z = swap (x, x + 1)

:sprint z

z = _

In [7]:
seq z ()

:sprint z

()

z = (_,_)

↑今度はうまくタプルの構成子が見えるところまで評価された

In [8]:
seq x ()

:sprint z

()

z = (_,3)

↑`x`だけ評価すると`3`が見えるようになった。`z`が評価されていないので`x + 1`の部分はサンク`_`のままである

今度はリストと `map` を使ったときの挙動を見てみる

In [9]:
xs = map (+1) [1..10] :: [Int]

:sprint xs

xs = _

In [10]:
seq xs ()

:sprint xs

()

xs = _ : _

↑`:`はリストの構成子なのでここが弱頭部正規形

In [11]:
length xs

:sprint xs

10

xs = [_,_,_,_,_,_,_,_,_,_]

↑`length`は中身を評価しないので構造だけが評価されて中はサンクのまま

In [12]:
sum xs

:sprint xs

65

xs = [2,3,4,5,6,7,8,9,10,11]

↑これで全部評価された

`:sprint` を使うとHaskellの遅延評価で何がどこまで評価されるのかインタラクティブに分かるので面白い。